In [1]:
import numpy as np
import os
import time
from keras import backend as K
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model, load_model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Input
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
def save_model_weights(model,path):
    model.save_weights(path)
    return
def save_model_arch(model,path):
    model.save(path)

In [4]:
initial_model = applications.VGG19(weights="imagenet", include_top=True)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
initial_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 224, 224, 64)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 224, 224, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 112, 112, 64)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [6]:
x = Dense(2, activation='softmax',name='classifier')(initial_model.layers[-2].output)
tune_model = Model(initial_model.input, x)
opt = Adam(lr=0.01)
tune_model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#sparse_categorical_crossentropy

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
tune_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 224, 224, 64)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 224, 224, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 112, 112, 64)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [8]:
path = 'data/'
target_size = (224, 224)
batch_size = 50
def get_batches(directory, target_size=target_size, batch_size=batch_size, shuffle=False):
 datagen = ImageDataGenerator()
 return datagen.flow_from_directory(directory=directory,
                                    target_size=target_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=shuffle)
batches = get_batches(path+'train', shuffle=True)
valid_batches = get_batches(path+'validation', batch_size=batch_size*2, shuffle=False)


Found 43 images belonging to 2 classes.
Found 41 images belonging to 2 classes.


In [9]:
out=tune_model.fit_generator(batches, samples_per_epoch=batches.nb_sample,nb_epoch=3, 
                         validation_data=valid_batches,
                         nb_val_samples=valid_batches.nb_sample)


Epoch 1/3
43/43 [==============================] - 51s - loss: 1.2062 - acc: 0.4651 - val_loss: 5.8969 - val_acc: 0.6341
Epoch 2/3
43/43 [==============================] - 47s - loss: 5.9974 - acc: 0.6279 - val_loss: 5.8969 - val_acc: 0.6341
Epoch 3/3
43/43 [==============================] - 47s - loss: 5.9974 - acc: 0.6279 - val_loss: 5.8969 - val_acc: 0.6341
